# Beginning our data-driven journey in Maji Ndogo

# Establishing a Connection to the MySQL Database

In [1]:
import json
import pandas as pd
import mysql.connector
from tabulate import tabulate

# Load database credentials from config.json
def load_config():
    with open("config.json", "r") as file:
        return json.load(file)

# Load the configuration
config = load_config()

# Function to create a connection to the MySQL database
def create_connection():
    connection = mysql.connector.connect(
        host=config["DB_HOST"],      
        user=config["DB_USER"],      
        password=config["DB_PASSWORD"],  
        database=config["DB_NAME"]      
    )
    return connection

# Create a connection object
conn = create_connection()

# Check the connection status
if conn.is_connected():
    print("Successfully connected to the database")
else:
    print("Failed to connect to the database")



Successfully connected to the database


# Get to know the Data

## View All Tables Present

In [2]:
# Function to execute a query and return results as a DataFrame
def execute_query(query):
    conn = create_connection()
    cursor = conn.cursor()
    cursor.execute(query)
    
    # Fetch all results
    results = cursor.fetchall()
    
    # Get column names from cursor description
    column_names = [desc[0] for desc in cursor.description]
    
    # Close cursor and connection
    cursor.close()
    conn.close()
    
    # Create a DataFrame from the fetched results
    df = pd.DataFrame(results, columns=column_names)
    return df

# Function to color headers with a custom style
def color_header():
    return [{'selector': 'thead th', 'props': [('background-color', 'lightblue'),
                                               ('color', 'black'),
                                               ('font-size', '12pt'),
                                               ('text-align', 'center')]}]

# Function to apply cell formatting for individual DataFrame cells
def highlight_cells(val):
    return 'background-color: #f4f4f4'  # Light grey for cell background

# Query to show all tables in the current database
query = "SHOW TABLES;"
tables_df = execute_query(query)

# Rename the column to 'Table Name'
tables_df.columns = ['Table Name']

# Apply custom table styles and display the styled DataFrame
styled_tables_df = (
    tables_df.style
    .applymap(highlight_cells)  # Apply custom function to highlight cells
    .set_table_styles(color_header())  # Apply custom header style
)

# Display the DataFrame with styles
print("Tables in the current database:")
display(styled_tables_df)

Tables in the current database:


,Table Name
0,data_dictionary
1,employee
2,global_water_access
3,location
4,visits
5,water_quality
6,water_source
7,well_pollution


## Displaying Sample Data from All Tables in the Database

In [3]:
# Function to color headers with a custom style
def color_header():
    return [{'selector': 'thead th', 'props': [('background-color', 'lightblue'),
                                               ('color', 'black'),
                                               ('font-size', '12pt'),
                                               ('text-align', 'center')]}]

# Function to apply cell formatting for individual DataFrame cells
def highlight_cells(val):
    return 'background-color: #f4f4f4'  # Light grey for cell background

# Get all tables in the database
tables_query = "SHOW TABLES;"
tables_df = execute_query(tables_query)

# Rename the column for better clarity
tables_df.columns = ['Table Name']

# Iterate through each table and display the first 10 rows as a styled DataFrame
for index, row in tables_df.iterrows():
    table_name = row['Table Name']  # Extract the table name from the DataFrame
    print(f"\nFirst 10 rows from table: {table_name}")
    
    # Query to fetch the first 10 rows from the current table
    query = f"SELECT * FROM {table_name} LIMIT 10;"
    
    # Execute the query to fetch the first 10 rows as a DataFrame
    table_df = execute_query(query)
    
    # Apply custom table styles and display the styled DataFrame
    styled_table = (
        table_df.style
        .applymap(highlight_cells)  # Apply custom function to highlight cells
        .set_table_styles(color_header())  # Apply custom header style
    )
    
    # Display the styled table
    display(styled_table)


First 10 rows from table: data_dictionary


,table_name,column_name,description,datatype,related_to
0,employee,assigned_employee_id,Unique ID assigned to each employee,INT,visits
1,employee,employee_name,Name of the employee,VARCHAR(255),
2,employee,phone_number,Contact number of the employee,VARCHAR(15),
3,employee,email,Email address of the employee,VARCHAR(255),
4,employee,address,Residential address of the employee,VARCHAR(255),
5,employee,town_name,Name of the town where the employee resides,VARCHAR(255),
6,employee,province_name,Name of the province where the employee resides,VARCHAR(255),
7,employee,position,Position or job title of the employee,VARCHAR(255),
8,visits,record_id,Unique ID assigned to each visit,int,"water_quality, water_source"
9,visits,location_id,ID of the location visited,varchar(255),location



First 10 rows from table: employee


,assigned_employee_id,employee_name,phone_number,email,address,province_name,town_name,position
0,0,Amara Jengo,+99637993287,None,36 Pwani Mchangani Road,Sokoto,Ilanga,Field Surveyor
1,1,Bello Azibo,+99643864786,None,129 Ziwa La Kioo Road,Kilimani,Rural,Field Surveyor
2,2,Bakari Iniko,+99222599041,None,18 Mlima Tazama Avenue,Hawassa,Rural,Field Surveyor
3,3,Malachi Mavuso,+99945849900,None,100 Mogadishu Road,Akatsi,Lusaka,Field Surveyor
4,4,Cheche Buhle,+99381679640,None,1 Savanna Street,Akatsi,Rural,Field Surveyor
5,5,Zuriel Matembo,+99034075111,None,26 Bahari Ya Faraja Road,Kilimani,Rural,Field Surveyor
6,6,Deka Osumare,+99379364631,None,104 Kenyatta Street,Akatsi,Rural,Field Surveyor
7,7,Lalitha Kaburi,+99681623240,None,145 Sungura Amanpour Road,Kilimani,Rural,Field Surveyor
8,8,Enitan Zuri,+99248509202,None,117 Kampala Road,Hawassa,Zanzibar,Field Surveyor
9,10,Farai Nia,+99570082739,None,33 Angélique Kidjo Avenue,Amanzi,Dahabu,Field Surveyor



First 10 rows from table: global_water_access


...

,record_id,subjective_quality_score,visit_count
0,0,0,1
1,1,1,1
2,2,5,1
3,3,10,1
4,4,4,1
5,5,0,1
6,6,9,1
7,7,10,1
8,8,2,1
9,9,10,1



First 10 rows from table: water_source


,source_id,type_of_water_source,number_of_people_served
0,AkHa00000224,tap_in_home,956
1,AkHa00001224,tap_in_home_broken,930
2,AkHa00002224,tap_in_home_broken,486
3,AkHa00003224,well,364
4,AkHa00004224,tap_in_home_broken,942
5,AkHa00005224,tap_in_home,736
6,AkHa00006224,tap_in_home,882
7,AkHa00007224,tap_in_home,554
8,AkHa00008224,well,398
9,AkHa00009224,well,346



First 10 rows from table: well_pollution


,source_id,date,description,pollutant_ppm,biological,results
0,KiRu28935224,2021-01-04 09:17:00,Bacteria: Giardia Lamblia,0.000000,495.898000,Contaminated: Biological
1,AkLu01628224,2021-01-04 09:53:00,Bacteria: E. coli,0.000000,6.096080,Contaminated: Biological
2,HaZa21742224,2021-01-04 10:37:00,"Inorganic contaminants: Zinc, Zinc, Lead, Cadmium",2.715000,0.000000,Contaminated: Chemical
3,HaRu19725224,2021-01-04 11:04:00,Clean,0.028859,0.000096,Clean
4,SoRu35703224,2021-01-04 11:29:00,Bacteria: E. coli,0.000000,22.500900,Contaminated: Biological
5,AkHa00070224,2021-01-04 11:42:00,Inorganic contaminants: Cadmium,5.467390,0.000000,Contaminated: Chemical
6,HaSe21346224,2021-01-04 11:52:00,Clean,0.014038,0.000090,Clean
7,HaYa21468224,2021-01-04 12:03:00,"Inorganic contaminants: Chromium, Barium, Chromium, Lead",6.051370,0.000000,Contaminated: Chemical
8,SoRu36278224,2021-01-04 12:24:00,Parasite: Cryptosporidium,0.000000,485.162000,Contaminated: Biological
9,AkLu02155224,2021-01-04 12:29:00,"Inorganic contaminants: Selenium, Arsenic",7.641060,0.000000,Contaminated: Chemical


## Dive into Water Sources: Finding Unique Water Source Types

In [4]:
# Function to color headers with a custom style
def color_header():
    return [{'selector': 'thead th', 'props': [('background-color', 'lightblue'),
                                               ('color', 'black'),
                                               ('font-size', '12pt'),
                                               ('text-align', 'center')]}]

# Function to apply cell formatting for individual DataFrame cells
def highlight_cells(val):
    return 'background-color: #f4f4f4'  # Light grey for cell background

# Query to find all unique water source types
query = "SELECT DISTINCT type_of_water_source FROM water_source;"

# Execute the query and fetch results as a DataFrame
unique_sources_df = execute_query(query)

# Apply custom table styles and display the styled DataFrame
styled_unique_sources = (
    unique_sources_df.style
    .applymap(highlight_cells)  # Apply custom function to highlight cells
    .set_table_styles(color_header())  # Apply custom header style
)

# Display the unique water source types DataFrame with styles
print("Unique water source types:")
display(styled_unique_sources)


Unique water source types:


,type_of_water_source
0,tap_in_home
1,tap_in_home_broken
2,well
3,shared_tap
4,river


## Unpack Visits to Water Sources: Locations Wheere queue time is over 8 hours

In [5]:
# Function to color headers with a custom style
def color_header():
    return [{'selector': 'thead th', 'props': [('background-color', 'lightblue'),
                                               ('color', 'black'),
                                               ('font-size', '12pt'),
                                               ('text-align', 'center')]}]

# Function to apply cell formatting for individual DataFrame cells
def highlight_cells(val):
    return 'background-color: #f4f4f4'  # Light grey for cell background

# SQL query to retrieve records from the visits table where time_in_queue > 500 minutes
query = """
SELECT 
    record_id, 
    location_id, 
    source_id, 
    time_of_record, 
    visit_count, 
    time_in_queue, 
    assigned_employee_id
FROM 
    visits
WHERE 
    time_in_queue > 500 LIMIT 10;
"""

# Execute the query and fetch results as a DataFrame
df_long_queue_times = execute_query(query)

# Display the DataFrame with custom styles
styled_df = df_long_queue_times.style.applymap(highlight_cells).set_table_styles(color_header())

# Show the styled DataFrame
print("Records with time_in_queue > 500 minutes (over 8 hours):")
styled_df

Records with time_in_queue > 500 minutes (over 8 hours):


,record_id,location_id,source_id,time_of_record,visit_count,time_in_queue,assigned_employee_id
0,899,SoRu35083,SoRu35083224,2021-01-16 10:14:00,6,515,28
1,2304,SoKo33124,SoKo33124224,2021-02-06 07:53:00,5,512,16
2,2315,KiRu26095,KiRu26095224,2021-02-06 14:32:00,3,529,8
3,3206,SoRu38776,SoRu38776224,2021-02-20 15:03:00,5,509,46
4,3701,HaRu19601,HaRu19601224,2021-02-27 12:53:00,3,504,0
5,4154,SoRu38869,SoRu38869224,2021-03-06 10:44:00,2,533,24
6,5483,AmRu14089,AmRu14089224,2021-03-27 18:15:00,4,509,12
7,9177,SoRu37635,SoRu37635224,2021-05-22 18:48:00,2,515,1
8,9648,SoRu36096,SoRu36096224,2021-05-29 11:24:00,2,533,3
9,11631,AkKi00881,AkKi00881224,2021-06-26 06:15:00,6,502,32


## Assess the quality of water sources:

In [6]:
# Function to color headers with a custom style
def color_header():
    return [{'selector': 'thead th', 'props': [('background-color', 'lightblue'),
                                               ('color', 'black'),
                                               ('font-size', '12pt'),
                                               ('text-align', 'center')]}]

# Function to apply cell formatting for individual DataFrame cells
def highlight_cells(val):
    return 'background-color: #f4f4f4'  # Light grey for cell background

# SQL query to find records with subjective_quality_score of 10 for home taps 
# and where the source was visited more than once
query = """
SELECT v.record_id, v.location_id, v.source_id, v.time_of_record, 
       wq.subjective_quality_score, wq.visit_count, v.time_in_queue, 
       v.assigned_employee_id
FROM visits v
JOIN water_quality wq ON v.record_id = wq.record_id
WHERE wq.subjective_quality_score = 10
  AND wq.visit_count > 1 
LIMIT 10;  -- Looking for records with visit count greater than 1
"""

# Execute the query and fetch results as a DataFrame
try:
    high_quality_home_taps = execute_query(query)

    # Display the DataFrame with custom styles
    styled_df = high_quality_home_taps.style.applymap(highlight_cells).set_table_styles(color_header())

    # Show the styled DataFrame
    print("Records where subjective_quality_score is 10 and visited more than once:")
    display(styled_df)  # Use display() to render the styled DataFrame properly in a Jupyter Notebook

except Exception as e:
    print(f"An error occurred: {e}")

Records where subjective_quality_score is 10 and visited more than once:


,record_id,location_id,source_id,time_of_record,subjective_quality_score,visit_count,time_in_queue,assigned_employee_id
0,59,SoRu35008,SoRu35008224,2021-01-02 07:27:00,10,2,248,1
1,67,SoRu35008,SoRu35008224,2021-01-03 14:31:00,10,3,104,1
2,85,SoRu35008,SoRu35008224,2021-01-04 12:12:00,10,4,165,1
3,128,SoRu35008,SoRu35008224,2021-01-05 08:13:00,10,5,213,1
4,137,KiMr25211,KiMr25211224,2021-01-05 09:47:00,10,2,64,3
5,232,KiMr25211,KiMr25211224,2021-01-06 12:34:00,10,3,110,3
6,263,SoRu35008,SoRu35008224,2021-01-06 15:48:00,10,6,93,1
7,269,AkRu08557,AkRu08557224,2021-01-06 18:55:00,10,2,48,1
8,271,AkRu08557,AkRu08557224,2021-01-07 07:11:00,10,3,99,1
9,317,KiMr25211,KiMr25211224,2021-01-07 12:52:00,10,4,57,3


## Investigate pollution issues:

In [7]:
# Function to color headers with a custom style
def color_header():
    return [{'selector': 'thead th', 'props': [('background-color', 'lightblue'),
                                               ('color', 'black'),
                                               ('font-size', '12pt'),
                                               ('text-align', 'center')]}]

# Function to apply cell formatting for individual DataFrame cells
def highlight_cells(val):
    return 'background-color: #f4f4f4'  # Light grey for cell background

# Define the query to retrieve pollution data
query = """
SELECT source_id, date, description, pollutant_ppm, biological, results
FROM well_pollution
LIMIT 10;
"""

# Execute the query and fetch results
try:
    pollution_data = execute_query(query)

    # Display the DataFrame with custom styles
    styled_df = pollution_data.style.applymap(highlight_cells).set_table_styles(color_header())

    # Show the styled DataFrame
    print("First few rows of pollution data:")
    display(styled_df)  # Use display() to render the styled DataFrame properly in a Jupyter Notebook

except Exception as e:
    print(f"An error occurred: {e}")

First few rows of pollution data:


,source_id,date,description,pollutant_ppm,biological,results
0,KiRu28935224,2021-01-04 09:17:00,Bacteria: Giardia Lamblia,0.000000,495.898000,Contaminated: Biological
1,AkLu01628224,2021-01-04 09:53:00,Bacteria: E. coli,0.000000,6.096080,Contaminated: Biological
2,HaZa21742224,2021-01-04 10:37:00,"Inorganic contaminants: Zinc, Zinc, Lead, Cadmium",2.715000,0.000000,Contaminated: Chemical
3,HaRu19725224,2021-01-04 11:04:00,Clean,0.028859,0.000096,Clean
4,SoRu35703224,2021-01-04 11:29:00,Bacteria: E. coli,0.000000,22.500900,Contaminated: Biological
5,AkHa00070224,2021-01-04 11:42:00,Inorganic contaminants: Cadmium,5.467390,0.000000,Contaminated: Chemical
6,HaSe21346224,2021-01-04 11:52:00,Clean,0.014038,0.000090,Clean
7,HaYa21468224,2021-01-04 12:03:00,"Inorganic contaminants: Chromium, Barium, Chromium, Lead",6.051370,0.000000,Contaminated: Chemical
8,SoRu36278224,2021-01-04 12:24:00,Parasite: Cryptosporidium,0.000000,485.162000,Contaminated: Biological
9,AkLu02155224,2021-01-04 12:29:00,"Inorganic contaminants: Selenium, Arsenic",7.641060,0.000000,Contaminated: Chemical


### a query that checks if the results is Clean but the biological column is > 0.01.

In [8]:
# Function to color headers with a custom style
def color_header():
    return [{'selector': 'thead th', 'props': [('background-color', 'lightblue'),
                                               ('color', 'black'),
                                               ('font-size', '12pt'),
                                               ('text-align', 'center')]}]

# Function to apply cell formatting for individual DataFrame cells
def highlight_cells(val):
    return 'background-color: #f4f4f4'  # Light grey for cell background

# Query to find records with Clean descriptions and biological > 0.01
query = """
SELECT *
FROM well_pollution
WHERE description LIKE 'Clean%' AND biological > 0.01 LIMIT 10;
"""

# Execute the query and get the results
results_df = execute_query(query)

# Apply styling to the DataFrame for display (if using Jupyter Notebook)
styled_df = results_df.style.applymap(highlight_cells).set_table_styles(color_header())
styled_df  # Display the styled DataFrame


,source_id,date,description,pollutant_ppm,biological,results
0,AkRu06489224,2021-01-10 09:44:00,Clean Bacteria: Giardia Lamblia,0.089790,38.467000,Clean
1,KiRu25473224,2021-02-07 15:51:00,Clean Bacteria: Giardia Lamblia,0.063009,24.453600,Clean
2,HaRu17401224,2021-03-01 13:44:00,Clean Bacteria: Giardia Lamblia,0.064921,25.812900,Clean
3,AkRu07137224,2021-03-04 13:41:00,Clean Bacteria: Giardia Lamblia,0.065684,18.297800,Clean
4,KiRu27205224,2021-03-13 14:17:00,Clean Bacteria: Giardia Lamblia,0.041802,49.428100,Clean
5,AkHa00514224,2021-04-11 12:11:00,Clean Bacteria: Giardia Lamblia,0.030540,22.025500,Clean
6,AmAm09776224,2021-05-23 11:28:00,Clean Bacteria: Giardia Lamblia,0.096382,13.657400,Clean
7,SoIl32894224,2021-07-11 11:37:00,Clean Bacteria: Giardia Lamblia,0.071241,5.449570,Clean
8,AkRu07366224,2021-07-23 11:19:00,Clean Bacteria: Giardia Lamblia,0.096946,26.030800,Clean
9,KiHa23443224,2021-09-05 12:34:00,Clean Bacteria: Giardia Lamblia,0.082800,13.716200,Clean
